<h1 align="center">
  <b>Universidad Autónoma de Chihuahua</b>
</h1>
<h2 align="center">
  <b>Facultad de Ingeniaría</b>
</h2>
<br>
<p align = "center">
  <a href="https://sega.uach.mx/">
     <img src="https://drive.google.com/uc?id=1n8NdPSF4WAZRxVomm74jf0zLU9ibdxqT">
  </a>
</p>

<h1 align="center">
  <b>Natural Language Processing with RNNs</b>
</h1>
<br>

<h2 align="center">
  <b>Data Science</b>
</h2>

<h3 align="center">
  <b>Proyecto Tercer Parcial</b>
</h3>
<br>
<p align="center">
<b>Link GitHub</b><br>
<href> </href>
</p>
<p align="center">
  <b>Alumnos: </b><br>
  Juan Luis Del Valle Sotelo - 338912 
  <br>
  Valeria Sofía Nevárez Juárez - 338811
</p>
<p align="center">
  <b>Profesor:</b>
  Jesús Roberto López Santillán
</p>
<br>



<p align="left">
  A miércoles 24 de mayo de 2023
</p>

In [ ]:
import sys
assert sys.version_info >= (3, 5)

IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB:
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers

import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


## Loading the Data and Preparing the Dataset

In [ ]:
file_path = "south_park.txt" 

In [ ]:
with open(file_path, 'r') as file:
    file_contents = file.read()

In [ ]:
unwanted_chars = "àáâäèéëíîñóôöúüабйнтщў؟آابتثخدرزشعفلمنهوچکی™"

translation_table = str.maketrans("", "", unwanted_chars)
south_park_text = file_contents.translate(translation_table)


In [ ]:
print(south_park_text[:200])

Stan 
You guys, you guys! Chef is going away. 

Kyle
Going away? For how long?

Stan
Forever.

Chef
I'm sorry boys.

Stan
Chef said he's been bored, so he joining a group called the Super Adventure Cl


In [ ]:
"".join(sorted(set(south_park_text.lower())))

"\t\n !#$%&'()*+,-./0123456789:;=?@_abcdefghijklmnopqrstuvwxyz¡¿’"

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(south_park_text)

In [ ]:
tokenizer.texts_to_sequences(["First"])

[[25, 7, 9, 8, 3]]

In [ ]:
tokenizer.sequences_to_texts([[25, 7, 9, 8, 3]])

['f i r s t']

In [ ]:
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([south_park_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_length = n_steps + 1 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [ ]:
dataset = dataset.prefetch(1)

In [ ]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 62) (32, 100)


## Creating and Training the Model

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=15)

Epoch 1/15
38327/38327 [==============================] - 606s 16ms/step - loss: 1.5378
Epoch 2/15
38327/38327 [==============================] - 525s 14ms/step - loss: 1.4552
Epoch 3/15
38327/38327 [==============================] - 540s 14ms/step - loss: 1.4349
Epoch 4/15
38327/38327 [==============================] - 541s 14ms/step - loss: 1.4246
Epoch 5/15
38327/38327 [==============================] - 533s 14ms/step - loss: 1.4184
Epoch 6/15
38327/38327 [==============================] - 535s 14ms/step - loss: 1.4137
Epoch 7/15
38327/38327 [==============================] - 528s 14ms/step - loss: 1.4103
Epoch 8/15
38327/38327 [==============================] - 552s 14ms/step - loss: 1.4072
Epoch 9/15
38327/38327 [==============================] - 534s 14ms/step - loss: 1.4046
Epoch 10/15
38327/38327 [==============================] - 526s 14ms/step - loss: 1.4027
Epoch 11/15
38327/38327 [==============================] - 526s 14ms/step - loss: 1.4010
Epoch 12/15
38327/38327 [=====

## Using the Model to Generate Text

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] 

'u'

In [ ]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [ ]:
def complete_text(text, n_chars=1000, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
tf.random.set_seed(42)

print(complete_text("t"))

t i should help and together help!  kick you mysterion. branding them about what i think that you gotuen anymore, complete you guys?  tupperwar, and soon, is dead weth unshou sesurer tacklite because etoses sack.  or, in a stopping crilnd of the coon. i actually it feel lekens! the wrong is, he're actually even destroy factices! but the coon and friends how cthulhu is just started people is wrong?!

coon
look, what should i do? 

singer
right, fucking guy!

toolshed
aww! where happened?! 

mosquito
yeah, no. 

liane
yes, because it's my undically grautan' shop even and coon, 'cause i'll totally be done into another dimension. and an the gleas rainbow.

coon
what do you mean as this curse? be should we do?

memt reporter
coon one reamonion. without him.

toolshed
all when we're gonna die-

woman k
toolshed!

toolshed
gotard! dude. you soon, mysterion! let's go home! 

red goth
i'll took dembsibey, per-that evil called owe my name!

loane
so the dark lord age!

coon
no! baby ap
lot right

## Stateful RNN

In [ ]:
tf.random.set_seed(42)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
383/383 [==============================] - 11s 21ms/step - loss: 2.6746
Epoch 2/50
383/383 [==============================] - 6s 15ms/step - loss: 2.2306
Epoch 3/50
383/383 [==============================] - 6s 16ms/step - loss: 2.0814
Epoch 4/50
383/383 [==============================] - 8s 20ms/step - loss: 2.0028
Epoch 5/50
383/383 [==============================] - 6s 15ms/step - loss: 1.9514
Epoch 6/50
383/383 [==============================] - 8s 20ms/step - loss: 1.9133
Epoch 7/50
383/383 [==============================] - 6s 17ms/step - loss: 1.8860
Epoch 8/50
383/383 [==============================] - 7s 19ms/step - loss: 1.8636
Epoch 9/50
383/383 [==============================] - 6s 17ms/step - loss: 1.8470
Epoch 10/50
383/383 [==============================] - 6s 16ms/step - loss: 1.8314
Epoch 11/50
383/383 [==============================] - 8s 20ms/step - loss: 1.8184
Epoch 12/50
383/383 [==============================] - 7s 18ms/step - loss: 1.8071
Epoch 13/50


In [ ]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [ ]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [ ]:
tf.random.set_seed(42)

print(complete_text("cartman"))

cartman
the fuck here are to the walk, then you thanks to fig brand?

mr. mackey
get arout warchist you, buke, but, i'm just going to be of the night, and you grist soes to the las somebody aren't real?

cartman
isn't a little new of said's knowed to keep shut the dad, i'm gonna did it to really addew here, we had to deptine and inshur. i've way-

stan
what is that?!

butters
just to a speak is bexise your hark. i'm not sorry, they peet the thried is the country, and i moaas.

welly
how chen?! 

boy
mush, i'm gonna foll brywhide cartumate d-!

croid and now, we shopped that put of 'he jimmy's popvy party guy in showidnel big... 

louse 2
it's going on, really gonna kyle video the fact is cure. it's trying.

cartman
i have to think of the hoado?

liane
oh, let's in.

kyle
hww you don't keep me where!  you're doing!

butters
hello, don't you sool kyle! i'm neliing that?

cartman
dube it wouldn't you dead. eyee are writtle guy. i stand some of the makes and now lage!

cartman
mimsy, p
lat

# Conclusión




En resumen, las redes neuronales recurrentes (RNN) pueden mostrar y completar palabras, pero carecen de sentido gramatical. En este caso, se habría podido mejorar el rendimiento corriendo el modelo durante más épocas, pero se enfrentaron problemas de desconexión en Google Colab. A pesar de sus limitaciones, las RNN pueden ser útiles en contextos creativos o como punto de partida para generar texto más avanzado. Mejorar y refinar estos modelos en el futuro puede llevar a resultados más coherentes y comprensibles.